In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
pd.set_option('display.max_columns',1000)
import warnings
warnings. simplefilter(action='ignore', category=Warning) 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
gen = pd.read_csv('/kaggle/input/daily-power-generation-in-india-20172020/file.csv')
gen.head()

In [ ]:
states = pd.read_csv('/kaggle/input/daily-power-generation-in-india-20172020/State_Region_corrected.csv')
states.head()

In [ ]:
share = states.groupby('Region').agg('sum')
share = share.reset_index()
share

In [ ]:
fig=px.bar(states,
           x='State / Union territory (UT)',
           y='Area (km2)',
           hover_data=['Area (km2)'],
           color='State / Union territory (UT)',
           )
fig.update_layout(title_text='Statewise Area(km2)')
fig.show()

In [ ]:
fig=px.bar(states,
           x='State / Union territory (UT)',
           y='National Share (%)',
           hover_data=['National Share (%)'],
           color='State / Union territory (UT)',
           )
fig.update_layout(title_text='Statewise Generation Share (%)')
fig.show()

In [ ]:
fig=make_subplots(rows=2, cols=2, 
                  subplot_titles=('Regionwise Area Share', 'Regionwise Area Share',
                                  'Regionwise Production Share', 'Regionwise Production Share'),
                 specs=[[{"type": "bar"}, {"type": "pie"}],
                       [{"type": "bar"}, {"type": "pie"}]])

fig.add_trace(
    go.Bar(x=share['Region'],
           y=share['Area (km2)'],
           name='Area'),
    row=1, col=1
)
fig.add_trace(
    go.Pie(labels=share['Region'],
           values=share['Area (km2)']),
    row=1, col=2
)
fig.add_trace(
    go.Bar(x=share['Region'],
           y=share['National Share (%)'],
           name='Production Share'),
    row=2, col=1
)
fig.add_trace(
    go.Pie(labels=share['Region'],
           values=share['Area (km2)']),
    row=2, col=2
)

fig.show()

In [ ]:
gen.dtypes

The features, Thermal Generation Actual (in MU) and Thermal Generation Estimated (in MU), should have been a float value but are given as object. Also, Date is given as object data type which should have been of DateTime

In [ ]:
gen.isnull().sum()

Nuclear Generation Estimation and Nuclear Generation Actual columns consists on null values. From the overview of dataset, we can clearly see that null values are in rows where the Region is Eastern or NorthEastern. This is probably due to the face that there are no nuclear power plants in the eastern and northeastern regions of India. Those null values can be replaced by 0 since there is no power generation.


In [ ]:
for col in gen.columns:
    if 'Nuclear' in col:
        gen[col] = gen[col].fillna(0)
gen.head()

Lets convert the Thermal Generation Actual and Thermal Geneartion Estimated from object to numeric type and Date from object to Datetime type.

In [ ]:
gen['Thermal Generation Actual (in MU)'] = gen['Thermal Generation Actual (in MU)'].str.replace(',','')
gen['Thermal Generation Estimated (in MU)'] = gen['Thermal Generation Estimated (in MU)'].str.replace(',','')

In [ ]:
gen['Thermal Generation Actual (in MU)'] = gen['Thermal Generation Actual (in MU)'].apply(pd.to_numeric, errors='coerce')
gen['Thermal Generation Estimated (in MU)'] = gen['Thermal Generation Estimated (in MU)'].apply(pd.to_numeric, errors = 'coerce')
gen['Date'] = pd.to_datetime(gen['Date'], errors='ignore')
gen.head()

In [ ]:
actual_cols = []
for col in gen.columns:
    if 'Actual' in col:
        actual_cols.append(col)
gen['Total Generation Actual (in MU)'] = 0
for col in actual_cols:
    gen['Total Generation Actual (in MU)'] = gen['Total Generation Actual (in MU)'] + gen[col]
gen.head()

In [ ]:
estimated_col = []
for col in gen.columns:
    if 'Estimated' in col:
        estimated_col.append(col)
gen['Total Generation Estimated (in MU)'] = 0
for col in estimated_col:
    gen['Total Generation Estimated (in MU)'] = gen['Total Generation Estimated (in MU)'] + gen[col]
gen.head()

In [ ]:
gen['Net Surplus'] = gen['Total Generation Actual (in MU)'] - gen['Total Generation Estimated (in MU)']
gen.head()

In [ ]:
gen['Year'] = pd.DatetimeIndex(gen['Date']).year
gen['Month'] = pd.DatetimeIndex(gen['Date']).month

In [ ]:
gen['Surplus in Thermal Generation'] = gen['Thermal Generation Actual (in MU)'] - gen['Thermal Generation Estimated (in MU)']
gen['Surplus in Nuclear Generation'] = gen['Nuclear Generation Actual (in MU)'] - gen['Nuclear Generation Estimated (in MU)']
gen['Surplus in Hydro Generation'] = gen['Hydro Generation Actual (in MU)'] - gen['Hydro Generation Estimated (in MU)']

In [ ]:
gen['Thermal Generation (in %)'] = (gen['Thermal Generation Actual (in MU)']/gen['Total Generation Actual (in MU)'])*100
gen['Nuclear Generation (in %)'] = (gen['Nuclear Generation Actual (in MU)']/gen['Total Generation Actual (in MU)'])*100
gen['Hydro Generation (in %)'] = (gen['Hydro Generation Actual (in MU)']/gen['Total Generation Actual (in MU)'])*100

In [ ]:
gen = gen[['Date', 'Month', 'Year', 'Region', 'Thermal Generation Actual (in MU)', 'Thermal Generation Estimated (in MU)', 
           'Thermal Generation (in %)', 'Surplus in Thermal Generation', 'Nuclear Generation Actual (in MU)', 
           'Nuclear Generation Estimated (in MU)', 'Nuclear Generation (in %)', 'Surplus in Nuclear Generation', 
           'Hydro Generation Actual (in MU)', 'Hydro Generation Estimated (in MU)', 'Hydro Generation (in %)',
           'Surplus in Hydro Generation', 'Total Generation Actual (in MU)', 'Net Surplus']]

In [ ]:
gen.head()

In [ ]:
Northern = gen[gen['Region'] == 'Northern']
Western = gen[gen['Region'] == 'Western']
Southern = gen[gen['Region'] == 'Southern']
Eastern = gen[gen['Region'] == 'Eastern']
NE = gen[gen['Region'] == 'NorthEastern']

### Comparing the Production Capacity of Each of the 5 Regions

#### Thermal Generation (in MU)

In [ ]:
fig=go.Figure()

fig.add_trace(
    go.Scatter(x=Northern['Date'],
               y=Northern['Thermal Generation Actual (in MU)'],
               marker_color='rgb(0,0,204)',
               name='Northern Region')
)

fig.add_trace(
    go.Scatter(x=Western['Date'],
               y=Western['Thermal Generation Actual (in MU)'],
               marker_color='rgb(204,0,0)',
               name='Western Region')
)

fig.add_trace(
    go.Scatter(x=Southern['Date'],
               y=Southern['Thermal Generation Actual (in MU)'],
               marker_color='rgb(102,102,51)',
               name='Southern Region')
)

fig.add_trace(
    go.Scatter(x=Eastern['Date'],
               y=Eastern['Thermal Generation Actual (in MU)'],
               marker_color='rgb(102,153,153)',
               name='Eastern Region')
)

fig.add_trace(
    go.Scatter(x=NE['Date'],
               y=NE['Thermal Generation Actual (in MU)'],
               marker_color='rgb(0,204,153)',
               name='North-East Region')
)


fig.update_xaxes(rangeslider_visible=True,
                 rangeselector=dict(
                    buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                    ])
                                 )
                )

fig.update_layout(title='Thermal Generation Actual (in MU)')
fig.show()

As visible from the above plot, Western Region produces the most thermal energy whereas the NorthEastern Region produces the least thermal energy

#### Nuclear Generation

In [ ]:
fig=go.Figure()

fig.add_trace(
    go.Scatter(x=Northern['Date'],
               y=Northern['Nuclear Generation Actual (in MU)'],
               marker_color='rgb(0,0,204)',
               name='Northern Region')
)

fig.add_trace(
    go.Scatter(x=Western['Date'],
               y=Western['Nuclear Generation Actual (in MU)'],
               marker_color='rgb(204,0,0)',
               name='Western Region')
)

fig.add_trace(
    go.Scatter(x=Southern['Date'],
               y=Southern['Nuclear Generation Actual (in MU)'],
               marker_color='rgb(102,102,51)',
               name='Southern Region')
)

fig.add_trace(
    go.Scatter(x=Eastern['Date'],
               y=Eastern['Nuclear Generation Actual (in MU)'],
               marker_color='rgb(102,153,153)',
               name='Eastern Region')
)

fig.add_trace(
    go.Scatter(x=NE['Date'],
               y=NE['Nuclear Generation Actual (in MU)'],
               marker_color='rgb(0,204,153)',
               name='North-East Region')
)


fig.update_xaxes(rangeslider_visible=True,
                 rangeselector=dict(
                    buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                    ])
                                 )
                )

fig.update_layout(title='Nuclear Generation Actual (in MU)')
fig.show()

From the above plot, we can see that the Southern region is the leading producer of Nuclear energy whereas in NorthEast region there in no production. 

#### Hydro Generation

In [ ]:
fig=go.Figure()

fig.add_trace(
    go.Scatter(x=Northern['Date'],
               y=Northern['Hydro Generation Actual (in MU)'],
               marker_color='rgb(0, 0, 204)',
               name='Northern Region')
)

fig.add_trace(
    go.Scatter(x=Western['Date'],
               y=Western['Hydro Generation Actual (in MU)'],
               marker_color='rgb(204,0,0)',
               name='Western Region')
)

fig.add_trace(
    go.Scatter(x=Southern['Date'],
               y=Southern['Hydro Generation Actual (in MU)'],
               marker_color='rgb(102,102,51)',
               name='Southern Region')
)

fig.add_trace(
    go.Scatter(x=Eastern['Date'],
               y=Eastern['Hydro Generation Actual (in MU)'],
               marker_color='rgb(102,153,153)',
               name='Eastern Region')
)

fig.add_trace(
    go.Scatter(x=NE['Date'],
               y=NE['Hydro Generation Actual (in MU)'],
               marker_color='rgb(0,204,153)',
               name='North-East Region')
)


fig.update_xaxes(rangeslider_visible=True,
                 rangeselector=dict(
                    buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                    ])
                                 )
                )

fig.update_layout(title='Hydro Generation Actual (in MU)')
fig.show()

The Northern regions are the leading production of Hydro electricity probably because many rivers are present in the Northern India

#### Total Generation

In [ ]:
fig=go.Figure()

fig.add_trace(
    go.Scatter(x=Northern['Date'],
               y=Northern['Total Generation Actual (in MU)'],
               marker_color='rgb(0, 0, 204)',
               name='Northern Region')
)

fig.add_trace(
    go.Scatter(x=Western['Date'],
               y=Western['Total Generation Actual (in MU)'],
               marker_color='rgb(204,0,0)',
               name='Western Region')
)

fig.add_trace(
    go.Scatter(x=Southern['Date'],
               y=Southern['Total Generation Actual (in MU)'],
               marker_color='rgb(102,102,51)',
               name='Southern Region')
)

fig.add_trace(
    go.Scatter(x=Eastern['Date'],
               y=Eastern['Total Generation Actual (in MU)'],
               marker_color='rgb(102,153,153)',
               name='Eastern Region')
)

fig.add_trace(
    go.Scatter(x=NE['Date'],
               y=NE['Total Generation Actual (in MU)'],
               marker_color='rgb(0,204,153)',
               name='North-East Region')
)


fig.update_xaxes(rangeslider_visible=True,
                 rangeselector=dict(
                    buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                    ])
                                 )
                )

fig.update_layout(title='Total Actual Generation (in MU)')
fig.show()

Western India is the leading producer of electricity while the NorthEast parts are hugely lagging behind in production.

#### Monthly Power Generation Between September 2017 and March 2020

In [ ]:
monthly_generation = gen[['Month', 'Year', 'Total Generation Actual (in MU)']].groupby(['Year', 'Month']).agg(sum)
monthly_generation.style.background_gradient(cmap='Blues_r')

In [ ]:
monthly_generation = monthly_generation.reset_index(['Year','Month'])


In [ ]:
monthly_generation['Year'] = monthly_generation['Year'].astype(str)
monthly_generation['Month'] = monthly_generation['Month'].astype(str)

In [ ]:
monthly_generation['Month/Year'] = monthly_generation['Month'] + '/' + monthly_generation['Year']


In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=monthly_generation['Month/Year'],
               y=monthly_generation['Total Generation Actual (in MU)'],
               mode='lines+markers',
               marker_color='rgb(0, 0, 204)',
               name='Total Generation Actual (in MU) line')
)
fig.add_trace(
    go.Bar(x=monthly_generation['Month/Year'],
           y=monthly_generation['Total Generation Actual (in MU)'],
           marker_color='rgb(204, 0, 0)',
           name='Total Generation Actual (in MU) bar')
)

fig.show()

In [ ]:
Northern['Thermal Generation Actual daily difference (in MU)'] = Northern['Thermal Generation Actual (in MU)'].diff()
Southern['Thermal Generation Actual daily difference (in MU)'] = Southern['Thermal Generation Actual (in MU)'].diff()
Western['Thermal Generation Actual daily difference (in MU)'] = Western['Thermal Generation Actual (in MU)'].diff()
Eastern['Thermal Generation Actual daily difference (in MU)'] = Eastern['Thermal Generation Actual (in MU)'].diff()
NE['Thermal Generation Actual daily difference (in MU)'] = NE['Thermal Generation Actual (in MU)'].diff()

In [ ]:
Northern['Nuclear Generation Actual daily difference (in MU)'] = Northern['Nuclear Generation Actual (in MU)'].diff()
Southern['Nuclear Generation Actual daily difference (in MU)'] = Southern['Nuclear Generation Actual (in MU)'].diff()
Western['Nuclear Generation Actual daily difference (in MU)'] = Western['Nuclear Generation Actual (in MU)'].diff()
Eastern['Nuclear Generation Actual daily difference (in MU)'] = Eastern['Nuclear Generation Actual (in MU)'].diff()
NE['Nuclear Generation Actual daily difference (in MU)'] = NE['Nuclear Generation Actual (in MU)'].diff()

In [ ]:
Northern['Hydro Generation Actual daily difference (in MU)'] = Northern['Hydro Generation Actual (in MU)'].diff()
Southern['Hydro Generation Actual daily difference (in MU)'] = Southern['Hydro Generation Actual (in MU)'].diff()
Western['Hydro Generation Actual daily difference (in MU)'] = Western['Hydro Generation Actual (in MU)'].diff()
Eastern['Hydro Generation Actual daily difference (in MU)'] = Eastern['Hydro Generation Actual (in MU)'].diff()
NE['Hydro Generation Actual daily difference (in MU)'] = NE['Hydro Generation Actual (in MU)'].diff()

In [ ]:
Northern['Net Surplus'] = Northern['Net Surplus'].diff()
Southern['Net Surplus'] = Southern['Net Surplus'].diff()
Western['Net Surplus'] = Western['Net Surplus'].diff()
Eastern['Net Surplus'] = Eastern['Net Surplus'].diff()
NE['Net Surplus'] = NE['Net Surplus'].diff()

In [ ]:
Northern.fillna(0, inplace = True)
Southern.fillna(0, inplace = True)
Western.fillna(0, inplace = True)
Eastern.fillna(0, inplace=True)
NE.fillna(0, inplace = True)

#### Day on Day Change in Power Generation (Thermal)

In [ ]:
fig_one = go.Figure(go.Scatter(x=Northern['Date'],
               y=Northern['Thermal Generation Actual daily difference (in MU)'],
               marker_color='rgb(0, 0, 204)',
               name='Northern Region')
)
fig_one.update_xaxes(rangeslider_visible=True,
                 rangeselector=dict(
                    buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                    ])
                                 )
                )
fig_one.update_layout(yaxis=dict(range=[-150,150]), title_text='Northern Region')
fig_one.show()

fig_two = go.Figure(go.Scatter(x=Southern['Date'],
               y=Southern['Thermal Generation Actual daily difference (in MU)'],
               marker_color='rgb(0, 0, 204)',
               name='Southern Region')
)
fig_two.update_xaxes(rangeslider_visible=True,
                 rangeselector=dict(
                    buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                    ])
                                 )
                )
fig_two.update_layout(yaxis=dict(range=[-150,150]), title_text='Southern Region')
fig_two.show()

fig_three = go.Figure(go.Scatter(x=Western['Date'],
               y=Western['Thermal Generation Actual daily difference (in MU)'],
               marker_color='rgb(0, 0, 204)',
               name='Western Region')
)
fig_three.update_xaxes(rangeslider_visible=True,
                 rangeselector=dict(
                    buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                    ])
                                 )
                )
fig_three.update_layout(yaxis=dict(range=[-150,150]), title_text='Western Region')
fig_three.show()

fig_four = go.Figure(go.Scatter(x=Eastern['Date'],
               y=Eastern['Thermal Generation Actual daily difference (in MU)'],
               marker_color='rgb(0, 0, 204)',
               name='Eastern Region')
)
fig_four.update_xaxes(rangeslider_visible=True,
                 rangeselector=dict(
                    buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                    ])
                                 )
                )
fig_four.update_layout(yaxis=dict(range=[-150,150]), title_text='Eastern Region')
fig_four.show()

fig_five = go.Figure(go.Scatter(x=NE['Date'],
               y=NE['Thermal Generation Actual daily difference (in MU)'],
               marker_color='rgb(0, 0, 204)',
               name='NE Region')
)
fig_five.update_xaxes(rangeslider_visible=True,
                 rangeselector=dict(
                    buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                    ])
                                 )
                )
fig_five.update_layout(yaxis=dict(range=[-200,200]), title_text='NE Region')
fig_five.show()

#### Day on Day Change in Power Generation (Nuclear)

In [ ]:
fig_one = go.Figure(go.Scatter(x=Northern['Date'],
               y=Northern['Nuclear Generation Actual daily difference (in MU)'],
               marker_color='rgb(0, 0, 204)',
               name='Northern Region')
)
fig_one.update_xaxes(rangeslider_visible=True,
                 rangeselector=dict(
                    buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                    ])
                                 )
                )
fig_one.update_layout(yaxis=dict(range=[-150,150]), title_text='Northern Region')
fig_one.show()

fig_two = go.Figure(go.Scatter(x=Southern['Date'],
               y=Southern['Nuclear Generation Actual daily difference (in MU)'],
               marker_color='rgb(0, 0, 204)',
               name='Southern Region')
)
fig_two.update_xaxes(rangeslider_visible=True,
                 rangeselector=dict(
                    buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                    ])
                                 )
                )
fig_two.update_layout(yaxis=dict(range=[-150,150]), title_text='Southern Region')
fig_two.show()

fig_three = go.Figure(go.Scatter(x=Western['Date'],
               y=Western['Nuclear Generation Actual daily difference (in MU)'],
               marker_color='rgb(0, 0, 204)',
               name='Western Region')
)
fig_three.update_xaxes(rangeslider_visible=True,
                 rangeselector=dict(
                    buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                    ])
                                 )
                )
fig_three.update_layout(yaxis=dict(range=[-150,150]), title_text='Western Region')
fig_three.show()

fig_four = go.Figure(go.Scatter(x=Eastern['Date'],
               y=Eastern['Nuclear Generation Actual daily difference (in MU)'],
               marker_color='rgb(0, 0, 204)',
               name='Eastern Region')
)
fig_four.update_xaxes(rangeslider_visible=True,
                 rangeselector=dict(
                    buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                    ])
                                 )
                )
fig_four.update_layout(yaxis=dict(range=[-150,150]), title_text='Eastern Region')
fig_four.show()

fig_five = go.Figure(go.Scatter(x=NE['Date'],
               y=NE['Nuclear Generation Actual daily difference (in MU)'],
               marker_color='rgb(0, 0, 204)',
               name='NE Region')
)
fig_five.update_xaxes(rangeslider_visible=True,
                 rangeselector=dict(
                    buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                    ])
                                 )
                )
fig_five.update_layout(yaxis=dict(range=[-150,150]), title_text='NE Region')
fig_five.show()

#### Day on Day Change in Power Generation (Hydro)

In [ ]:
fig_one = go.Figure(go.Scatter(x=Northern['Date'],
               y=Northern['Hydro Generation Actual daily difference (in MU)'],
               marker_color='rgb(0, 0, 204)',
               name='Northern Region')
)
fig_one.update_xaxes(rangeslider_visible=True,
                 rangeselector=dict(
                    buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                    ])
                                 )
                )
fig_one.update_layout(yaxis=dict(range=[-150,150]), title_text='Northern Region')
fig_one.show()

fig_two = go.Figure(go.Scatter(x=Southern['Date'],
               y=Southern['Hydro Generation Actual daily difference (in MU)'],
               marker_color='rgb(0, 0, 204)',
               name='Southern Region')
)
fig_two.update_xaxes(rangeslider_visible=True,
                 rangeselector=dict(
                    buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                    ])
                                 )
                )
fig_two.update_layout(yaxis=dict(range=[-150,150]), title_text='Southern Region')
fig_two.show()

fig_three = go.Figure(go.Scatter(x=Western['Date'],
               y=Western['Hydro Generation Actual daily difference (in MU)'],
               marker_color='rgb(0, 0, 204)',
               name='Western Region')
)
fig_three.update_xaxes(rangeslider_visible=True,
                 rangeselector=dict(
                    buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                    ])
                                 )
                )
fig_three.update_layout(yaxis=dict(range=[-150,150]), title_text='Western Region')
fig_three.show()

fig_four = go.Figure(go.Scatter(x=Eastern['Date'],
               y=Eastern['Hydro Generation Actual daily difference (in MU)'],
               marker_color='rgb(0, 0, 204)',
               name='Eastern Region')
)
fig_four.update_xaxes(rangeslider_visible=True,
                 rangeselector=dict(
                    buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                    ])
                                 )
                )
fig_four.update_layout(yaxis=dict(range=[-150,150]), title_text='Eastern Region')
fig_four.show()

fig_five = go.Figure(go.Scatter(x=NE['Date'],
               y=NE['Hydro Generation Actual daily difference (in MU)'],
               marker_color='rgb(0, 0, 204)',
               name='NE Region')
)
fig_five.update_xaxes(rangeslider_visible=True,
                 rangeselector=dict(
                    buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                    ])
                                 )
                )
fig_five.update_layout(yaxis=dict(range=[-150,150]), title_text='NE Region')
fig_five.show()

For some reason, there is spike (higher or lower than previous days of month) of power generation on first day of every month

In [ ]:
fig_one = go.Figure(go.Scatter(x=Northern['Date'],
               y=Northern['Nuclear Generation (in %)'],
               marker_color='rgb(0, 0, 204)',
               name='Northern Region')
)
fig_one.update_xaxes(rangeslider_visible=True,
                 rangeselector=dict(
                    buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                    ])
                                 )
                )
fig_one.update_layout(yaxis=dict(range=[-10,10]), title_text='Northern Region')
fig_one.show()

fig_two = go.Figure(go.Scatter(x=Southern['Date'],
               y=Southern['Nuclear Generation (in %)'],
               marker_color='rgb(0, 0, 204)',
               name='Southern Region')
)
fig_two.update_xaxes(rangeslider_visible=True,
                 rangeselector=dict(
                    buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                    ])
                                 )
                )
fig_two.update_layout(yaxis=dict(range=[-15,15]), title_text='Southern Region')
fig_two.show()

fig_three = go.Figure(go.Scatter(x=Western['Date'],
               y=Western['Nuclear Generation (in %)'],
               marker_color='rgb(0, 0, 204)',
               name='Western Region')
)
fig_three.update_xaxes(rangeslider_visible=True,
                 rangeselector=dict(
                    buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                    ])
                                 )
                )
fig_three.update_layout(yaxis=dict(range=[-7,7]), title_text='Western Region')
fig_three.show()

fig_four = go.Figure(go.Scatter(x=Eastern['Date'],
               y=Eastern['Nuclear Generation (in %)'],
               marker_color='rgb(0, 0, 204)',
               name='Eastern Region')
)
fig_four.update_xaxes(rangeslider_visible=True,
                 rangeselector=dict(
                    buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                    ])
                                 )
                )
fig_four.update_layout(yaxis=dict(range=[-7,7]), title_text='Eastern Region')
fig_four.show()

fig_five = go.Figure(go.Scatter(x=NE['Date'],
               y=NE['Nuclear Generation (in %)'],
               marker_color='rgb(0, 0, 204)',
               name='NE Region')
)
fig_five.update_xaxes(rangeslider_visible=True,
                 rangeselector=dict(
                    buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                    ])
                                 )
                )
fig_five.update_layout(yaxis=dict(range=[-7,7]), title_text='NE Region')
fig_five.show()

The production of nuclear energy (in %) has remained more or less the same between these times.

In [ ]:
fig_one = go.Figure(go.Scatter(x=Northern['Date'],
               y=Northern['Thermal Generation (in %)'],
               marker_color='rgb(0, 0, 204)',
               name='Northern Region')
)
fig_one.update_xaxes(rangeslider_visible=True,
                 rangeselector=dict(
                    buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                    ])
                                 )
                )
fig_one.update_layout(yaxis=dict(range=[-100,100]), title_text='Northern Region')
fig_one.show()

fig_two = go.Figure(go.Scatter(x=Southern['Date'],
               y=Southern['Thermal Generation (in %)'],
               marker_color='rgb(0, 0, 204)',
               name='Southern Region')
)
fig_two.update_xaxes(rangeslider_visible=True,
                 rangeselector=dict(
                    buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                    ])
                                 )
                )
fig_two.update_layout(yaxis=dict(range=[-100,100]), title_text='Southern Region')
fig_two.show()

fig_three = go.Figure(go.Scatter(x=Western['Date'],
               y=Western['Thermal Generation (in %)'],
               marker_color='rgb(0, 0, 204)',
               name='Western Region')
)
fig_three.update_xaxes(rangeslider_visible=True,
                 rangeselector=dict(
                    buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                    ])
                                 )
                )
fig_three.update_layout(yaxis=dict(range=[-100,100]), title_text='Western Region')
fig_three.show()

fig_four = go.Figure(go.Scatter(x=Eastern['Date'],
               y=Eastern['Thermal Generation (in %)'],
               marker_color='rgb(0, 0, 204)',
               name='Eastern Region')
)
fig_four.update_xaxes(rangeslider_visible=True,
                 rangeselector=dict(
                    buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                    ])
                                 )
                )
fig_four.update_layout(yaxis=dict(range=[-100,100]), title_text='Eastern Region')
fig_four.show()

fig_five = go.Figure(go.Scatter(x=NE['Date'],
               y=NE['Thermal Generation (in %)'],
               marker_color='rgb(0, 0, 204)',
               name='NE Region')
)
fig_five.update_xaxes(rangeslider_visible=True,
                 rangeselector=dict(
                    buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                    ])
                                 )
                )
fig_five.update_layout(yaxis=dict(range=[-100,100]), title_text='NE Region')
fig_five.show()

Thermal energy production (in %) is maximum during the months of December, January whereas minimum in months of July,August,September

In [ ]:
fig_one = go.Figure(go.Scatter(x=Northern['Date'],
               y=Northern['Hydro Generation (in %)'],
               marker_color='rgb(0, 0, 204)',
               name='Northern Region')
)
fig_one.update_xaxes(rangeslider_visible=True,
                 rangeselector=dict(
                    buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                    ])
                                 )
                )
fig_one.update_layout(yaxis=dict(range=[-50,50]), title_text='Northern Region')
fig_one.show()

fig_two = go.Figure(go.Scatter(x=Southern['Date'],
               y=Southern['Hydro Generation (in %)'],
               marker_color='rgb(0, 0, 204)',
               name='Southern Region')
)
fig_two.update_xaxes(rangeslider_visible=True,
                 rangeselector=dict(
                    buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                    ])
                                 )
                )
fig_two.update_layout(yaxis=dict(range=[-50,50]), title_text='Southern Region')
fig_two.show()

fig_three = go.Figure(go.Scatter(x=Western['Date'],
               y=Western['Hydro Generation (in %)'],
               marker_color='rgb(0, 0, 204)',
               name='Western Region')
)
fig_three.update_xaxes(rangeslider_visible=True,
                 rangeselector=dict(
                    buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                    ])
                                 )
                )
fig_three.update_layout(yaxis=dict(range=[-50,50]), title_text='Western Region')
fig_three.show()

fig_four = go.Figure(go.Scatter(x=Eastern['Date'],
               y=Eastern['Hydro Generation (in %)'],
               marker_color='rgb(0, 0, 204)',
               name='Eastern Region')
)
fig_four.update_xaxes(rangeslider_visible=True,
                 rangeselector=dict(
                    buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                    ])
                                 )
                )
fig_four.update_layout(yaxis=dict(range=[-50,50]), title_text='Eastern Region')
fig_four.show()

fig_five = go.Figure(go.Scatter(x=NE['Date'],
               y=NE['Hydro Generation (in %)'],
               marker_color='rgb(0, 0, 204)',
               name='NE Region')
)
fig_five.update_xaxes(rangeslider_visible=True,
                 rangeselector=dict(
                    buttons=list([
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all")
                    ])
                                 )
                )
fig_five.update_layout(yaxis=dict(range=[-50,50]), title_text='NE Region')
fig_five.show()

Production of Hydro electricity (in %) is maximum in the month of August, September (Summer) while minimum in months of December, January (Winter)